# **Imports & Settings**

In [3]:
from utils_data_prep import *
import os
import numpy as np
import rasterio
from rasterio.plot import show
import whitebox
from scipy.ndimage import gaussian_filter

import matplotlib.pyplot as plt

In [4]:
#########################
# Initialize Whitebox
#########################
# USER MANUAL: https://whiteboxgeo.com/manual/wbt_book/preface.html

# import whitebox tools class as wbt object
wbt = whitebox.WhiteboxTools()

# toggle on/off geoprocessing tool outputs (optional)
# wbt.verbose = False

# set working directory for input/output files to current working directory
wbt.set_working_dir(os.getcwd())

# print whitebox version to verify correctly loading
wbt.version()

"WhiteboxTools v2.4.0 (c) Dr. John Lindsay 2017-2023\n\nWhiteboxTools is an advanced geospatial data analysis platform developed at\nthe University of Guelph's Geomorphometry and Hydrogeomatics Research \nGroup (GHRG). See www.whiteboxgeo.com for more details.\n"


# **Terrain Features**

***OVERVIEW***
* Terrain features are derived from the DEM to characterize the topography and capture features that are critical for identifying and delineating surficial geologic map units. These features provide supplementary data about topography, weathering processes, water drainage patterns, and vegetation that enhance the interpretation of geologic processes and landforms, contributing to more accurate predictions.

* Terrain features may be calculated using a variety of GIS tools. The features below use the Python implementation of Whitebox Tools. Details from the user manual are presented below each tool, and the reader is referred to the home page for more information - https://www.whiteboxgeo.com/manual/wbt_book/preface.html

***PURPOSE***
* Eight terrain features are calculated from the DEM, each providing unique insights into the landscape's characteristics:

    1. **Slope.** Measures the steepness or incline of the terrain. It is critical for identifying areas prone to erosion, landslides, or specific deposition patterns associated with certain geologic map units.

    2. **Profile Curvature.** Indicates the curvature of the terrain in the direction of the slope. It affects the acceleration or deceleration of water flow, which can influence erosion and sediment deposition patterns.

    3. **Planform Curvature.** Measures the curvature perpendicular to the slope direction. It helps in identifying convergent and divergent flow patterns, which are important for understanding water drainage and sediment deposition.

    4. **Standard Deviation of Slope.** Provides a measure of the variability in slope within a given area. It is useful for identifying areas with complex topography, which may correlate with different geologic processes or materials.

    5. **Elevation Percentile.** Measure of local tpi...

## *Multi-scale Analysis*

* Multiple scales of terrain features can be critical to accurately detect and characterize surficial geologic map units. 

* The DEM will be scaled to multiple resolutions and each of these will then be used to calculate the terrain features.

    * Gaussian filter (3x3 or 5x5 kernel) will be applied to the original DEM

    * The filtered DEM will then be downsampled to several lower resolutions to capture local, intermediate, and global scale topographic features that may be relevant to the ultimate goal of identifying surficial geologic map units.

        * Original DEM has a spatial resolution of 5x5 feet per pixel.

        * Downsample resolutions will be 10x10, 20x20, 100x100, and 200x200 feet per pixel.

    * Terrain features will be calculated directly from downsampled DEMs.
    
    * Scaled Terrain features will then be upsampled and aligned with the original DEM and other features in the dataset.

In [ ]:
from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform, reproject

def resample_image(input_path, new_resolution, output_path):

    with rasterio.open(input_path) as src:

        # calculate the new transform and dimensions based on the new resolution
        dst_transform, dst_width, dst_height = calculate_default_transform(src.crs, 
                                                                           src.crs, 
                                                                           src.width, 
                                                                           src.height, 
                                                                           *src.bounds, 
                                                                           resolution=new_resolution)
        
        dst_meta = src.meta.copy()
        dst_meta.update({'driver': 'GTiff', 
                         'width': dst_width, 
                         'height': dst_height, 
                         'transform':dst_transform})
        
        with rasterio.open(output_path, 'w', **dst_meta) as dst:

            reproject(source=rasterio.band(src, 1), 
                      destination=rasterio.band(dst, 1), 
                      src_transform=src.transform, 
                      src_crs=src.crs, 
                      dst_transform=dst_transform, 
                      dst_crs=src.crs, 
                      resampling=Resampling.cubic)

In [ ]:
def apply_gaussian_filter(input_path, sigma, output_path):

    with rasterio.open(input_path) as src:

        src_data = src.read(1, masked=True)

        dst_data = gaussian_filter(src_data, sigma=sigma)

        dst_meta = src.meta.copy()

    with rasterio.open(output_path, 'w', **dst_meta) as dst:
        dst.write(dst_data, 1)
    pass

In [ ]:
################################
# Create Multiple Scales of DEM
################################

# path to original dem
dem_path = r'../data/warren/dem_buffered.tif'

# list of resolution(s) to downsample DEM (in native spatial units, which is feet here)
downsample_resolutions = [10, 20, 100, 200]


##### downsample DEM and save new GeoTIFF images
for res in downsample_resolutions:
    output_path = f"../data/warren/dem_{res}.tif"
    downsample_image(dem_path, res, output_path)

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
with rasterio.open(r'../data/warren/dem_10.tif') as src:
    show(src, ax=ax, cmap='terrain')
ax.set_axis_off()
ax.set_xlim(4750000, 4770000)
ax.set_ylim(3540000, 3560000)
plt.tight_layout()
plt.show()

In [19]:
x = np.arange(9, dtype=np.float32).reshape((3,3))
x

array([[0., 1., 2.],
       [3., 4., 5.],
       [6., 7., 8.]], dtype=float32)

In [20]:
gaussian_filter(x, sigma=1)

array([[1.688295 , 2.2662213, 2.8441474],
       [3.4220738, 4.       , 4.577926 ],
       [5.1558523, 5.7337785, 6.3117046]], dtype=float32)

## *Slope*

This tool calculates slope gradient (i.e. slope steepness in degrees, radians, or percent) for each grid cell in an input digital elevation model (DEM). For DEMs in projected coordinate systems, the tool uses the 3rd-order bivariate Taylor polynomial method described by Florinsky (2016). Based on a polynomial fit of the elevations within the 5x5 neighbourhood surrounding each cell, this method is considered more robust against outlier elevations (noise) than other methods.

* https://www.whiteboxgeo.com/manual/wbt_book/available_tools/geomorphometric_analysis.html#Slope

In [ ]:
###########################
# Calculate Slope from DEM
###########################

# path to dem GeoTIFF
dem_path = r'../data/warren/dem.tif'

# path for output slope GeoTIFF
output_path = r'../data/warren/slope.tif'


##### calculate terrain feature

## *Profile Curvature*

This tool calculates the profile (or vertical) curvature, or the rate of change in slope along a flow line, from a digital elevation model (DEM). It is the curvature of a normal section having a common tangent line with a slope line at a given point on the surface (Florinsky, 2017). This variable has an unbounded range that can take either positive or negative values. Positive values of the index are indicative of flow acceleration while negative profile curvature values indicate flow deceleration. Profile curvature is measured in units of m-1. For DEMs in projected coordinate systems, the tool uses the 3rd-order bivariate Taylor polynomial method described by Florinsky (2016). Based on a polynomial fit of the elevations within the 5x5 neighbourhood surrounding each cell, this method is considered more robust against outlier elevations (noise) than other methods.

* https://www.whiteboxgeo.com/manual/wbt_book/available_tools/geomorphometric_analysis.html#ProfileCurvature

In [ ]:
#######################################
# Calculate Profile Curvature from DEM
#######################################

# path to dem GeoTIFF
dem_path = r'../data/warren/dem.tif'

# path for output slope GeoTIFF
output_path = r'../data/warren/profilecurvature.tif'


##### calculate terrain feature

## *Planform Curvature*

This tool calculates the plan (or contour) curvature from a digital elevation model (DEM). Plan curvature is the curvature of a contour line at a given point on the topographic surface (Florinsky, 2017). This variable has an unbounded range that can take either positive or negative values. Positive values of the index are indicative of flow divergence while negative plan curvature values indicate flow convergence. Thus plan curvature is similar to tangential curvature, although the literature suggests that the latter is more numerically stable (Wilson, 2018). Plan curvature is measured in units of m-1. method described by Florinsky (2016). Based on a polynomial fit of the elevations within the 5x5 neighbourhood surrounding each cell, this method is considered more robust against outlier elevations (noise) than other methods. 

* https://www.whiteboxgeo.com/manual/wbt_book/available_tools/geomorphometric_analysis.html#PlanCurvature

In [ ]:
#######################################
# Calculate Profile Curvature from DEM
#######################################

# path to dem GeoTIFF
dem_path = r'../data/warren/dem.tif'

# path for output slope GeoTIFF
output_path = r'../data/warren/plancurvature.tif'


##### calculate terrain feature

## *Standard Deviation of Slope*

Calculates the standard deviation of slope from an input DEM, a metric of roughness described by Grohmann et al., (2011).

* https://www.whiteboxgeo.com/manual/wbt_book/available_tools/geomorphometric_analysis.html#StandardDeviationOfSlope



In [ ]:
#################################################
# Calculate Standard Deviation of Slope from DEM
#################################################

# path to dem GeoTIFF
dem_path = r'../data/warren/dem.tif'

# window size(s) for calculating ep (in pixels)
windows = [5, 11]


##### calculate terrain feature
for window in windows:
    output_path = f"r../data/warren/sdslope_{window}.tif"

## *Elevation Percentile*

Elevation percentile (EP) is a measure of local topographic position (LTP). It expresses the vertical position for a digital elevation model (DEM) grid cell (z0) as the percentile of the elevation distribution within the filter window, such that:

EP = counti∈C(zi < z0) x (100 / nC)

where z0 is the elevation of the window's center grid cell, zi is the elevation of cell i contained within the neighboring set C, and nC is the number of grid cells contained within the window.

EP is unsigned and expressed as a percentage, bound between 0% and 100%. Quantile-based estimates (e.g., the median and interquartile range) are often used in nonparametric statistics to provide data variability estimates without assuming the distribution is normal. Thus, EP is largely unaffected by irregularly shaped elevation frequency distributions or by outliers in the DEM, resulting in a highly robust metric of LTP. In fact, elevation distributions within small to medium sized neighborhoods often exhibit skewed, multimodal, and non-Gaussian distributions, where the occurrence of elevation errors can often result in distribution outliers. Thus, based on these statistical characteristics, EP is considered one of the most robust representation of LTP.

* https://www.whiteboxgeo.com/manual/wbt_book/available_tools/geomorphometric_analysis.html#ElevPercentile

In [ ]:
##########################################
# Calculate Elevation Percentile from DEM
##########################################

# path to dem GeoTIFF
dem_path = r'../data/warren/dem.tif'

# window size(s) for calculating ep (in pixels)
windows = [5, 11]


##### calculate terrain feature
for window in windows:
    output_path = f"r../data/warren/ep_{window}.tif"

# **PCA Multi-Scale Features**

# **Align with Dataset**